In [ ]:
# Librerías necesarias
import numpy as np
import pickle, tarfile, os
from urllib.request import urlretrieve

# Función para cargar CIFAR-10
def load_CIFAR10(path):
    url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
    if not os.path.exists(path): os.makedirs(path)
    tgz_path = os.path.join(path, 'cifar-10-python.tar.gz')
    if not os.path.exists(tgz_path):
        print("Descargando CIFAR-10...")
        urlretrieve(url, tgz_path)
    with tarfile.open(tgz_path, 'r:gz') as tar:
        tar.extractall(path)
    def load_batch(file):
        with open(file, 'rb') as f:
            d = pickle.load(f, encoding='bytes')
            return d[b'data'], d[b'labels']
    Xs, Ys = [], []
    for i in range(1, 6):
        X, Y = load_batch(os.path.join(path, 'cifar-10-batches-py', f'data_batch_{i}'))
        Xs.append(X); Ys.append(Y)
    Xtr = np.concatenate(Xs); Ytr = np.concatenate(Ys)
    return Xtr, np.array(Ytr)

# Cargar datos
Xtr, Ytr = load_CIFAR10('data/cifar10/')



Descargando CIFAR-10...


In [ ]:
# Reducir tamaño para ejecución rápida
Xtr = Xtr[:400]
Ytr = Ytr[:400]

# Aplanar (ya está en 3072), agregar bias (columna de 1's)
Xtr = np.hstack([Xtr, np.ones((Xtr.shape[0], 1))])  # (400, 3073)

# Transponer a (3073, N)
Xtr = Xtr.T  # (3073, 400)


Loss (no vectorizado): 9.000326337987813
Loss (semi vectorizado): 9.000326337987811
Loss (total vectorizado): 9.000056318304365


In [ ]:
def L(X, y, W):
    delta = 1.0
    scores = W.dot(X)  # (C, N)
    correct_class_scores = scores[y, np.arange(X.shape[1])]  # (N,)
    margins = np.maximum(0, scores - correct_class_scores + delta)
    margins[y, np.arange(X.shape[1])] = 0
    return np.sum(margins) / X.shape[1]  # promedio de la pérdida



SVM Loss (CIFAR-10): 42.29002912983443


In [ ]:
# Inicializar pesos aleatorios pequeños
W = np.random.randn(10, 3073) * 0.001  # 10 clases, 3073 características

# Calcular pérdida
loss = L(Xtr, Ytr, W)
print("SVM Loss en CIFAR-10:", loss)


SVM Loss en CIFAR-10: 35.97577998080269
